In [1]:
import requests
from collections import defaultdict
from datetime import datetime, timedelta
from langchain.agents import Tool
from langchain.llms import LlamaCpp 
from langchain.agents import initialize_agent
import json
api_key = "c6dfc4d92a8f972d237ef696ec87b37a"

In [2]:
def get_weather_info(city):
    """Fetches current weather information for a city using OpenWeatherMap API."""
    
    url_current = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response_current = requests.get(url_current)
    if response_current.status_code != 200:
        return "Error: Could not fetch weather data."
    data_current = response_current.json()

    response = {
        'coordinates': data_current['coord'],
        'weather': data_current['weather'][0],
        'temperature': {
            'current': data_current['main']['temp'],
            'feels_like': data_current['main']['feels_like'],
            'min': data_current['main']['temp_min'],
            'max': data_current['main']['temp_max']
        },
        'pressure': {
            'sea_level': data_current['main'].get('sea_level', data_current['main']['pressure']),
            'ground_level': data_current['main'].get('grnd_level', data_current['main']['pressure'])
        },
        'humidity': data_current['main']['humidity'],
        'visibility': data_current['visibility'],
        'wind': data_current['wind'],
        'clouds': data_current['clouds'],
        'rain': data_current.get('rain', {}),
        'dt': data_current['dt'],
        'sys': data_current['sys'],
        'timezone': data_current['timezone'],
        'id': data_current['id'],
        'name': data_current['name'],
        'cod': data_current['cod']
    }

    return json.dumps(response, indent=2)

In [3]:
def get_forecast(city):
    """Fetches 5-day weather forecast for a city using OpenWeatherMap API."""
    url_forecast = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
    response_forecast = requests.get(url_forecast)
    if response_forecast.status_code != 200:
        return "Error: Could not fetch forecast data."
    forecast_json = response_forecast.json()
    
    return restructure_forecast(forecast_json)

In [4]:
def restructure_forecast(forecast_json):
    """Restructures the forecast JSON data into a nested dictionary by date and time, excluding the current date."""
    current_date = datetime.now().date()
    next_date = current_date + timedelta(days=1)

    structured_data = defaultdict(dict)
    
    for entry in forecast_json['list']:
        date, time = entry['dt_txt'].split()
        if date != str(current_date):
            structured_data[date][time] = {
                'temperature': entry['main']['temp'],
                'feels_like': entry['main']['feels_like'],
                'temp_min': entry['main']['temp_min'],
                'temp_max': entry['main']['temp_max'],
                'pressure': entry['main']['pressure'],
                'humidity': entry['main']['humidity'],
                'weather': entry['weather'][0]['description'],
                'icon': entry['weather'][0]['icon'],
                'wind_speed': entry['wind']['speed'],
                'wind_deg': entry['wind']['deg'],
                'visibility': entry['visibility'],
                'pop': entry['pop'],
                'rain': entry['rain']['3h'] if 'rain' in entry else 0,
                'clouds': entry['clouds']['all']
            }
    
    # Only return data for the next date
    return {str(next_date): structured_data[str(next_date)]}

In [5]:
city = "London"
forecast = get_forecast(city)
print(forecast)

{'2024-07-05': {'00:00:00': {'temperature': 12.86, 'feels_like': 12.21, 'temp_min': 12.86, 'temp_max': 12.86, 'pressure': 1011, 'humidity': 77, 'weather': 'scattered clouds', 'icon': '03n', 'wind_speed': 4.81, 'wind_deg': 240, 'visibility': 10000, 'pop': 0, 'rain': 0, 'clouds': 49}, '03:00:00': {'temperature': 12.4, 'feels_like': 11.78, 'temp_min': 12.4, 'temp_max': 12.4, 'pressure': 1011, 'humidity': 80, 'weather': 'broken clouds', 'icon': '04n', 'wind_speed': 3.5, 'wind_deg': 234, 'visibility': 10000, 'pop': 0, 'rain': 0, 'clouds': 70}, '06:00:00': {'temperature': 12.5, 'feels_like': 12.23, 'temp_min': 12.5, 'temp_max': 12.5, 'pressure': 1009, 'humidity': 93, 'weather': 'light rain', 'icon': '10d', 'wind_speed': 2.14, 'wind_deg': 183, 'visibility': 10000, 'pop': 0.57, 'rain': 0.77, 'clouds': 85}, '09:00:00': {'temperature': 15.13, 'feels_like': 15.21, 'temp_min': 15.13, 'temp_max': 15.13, 'pressure': 1008, 'humidity': 96, 'weather': 'light rain', 'icon': '10d', 'wind_speed': 4.03, 'w

In [6]:
# Weather Tool
weather_tool = Tool(
    name="WeatherLookup",
    func=get_weather_info,
    description="Useful to get the current weather (Today) information for a city. It includes information on temperature, pressure, humidity, wind, clouds and rain."
)

# Forecast Tool
forecast_tool = Tool(
    name="ForecastLookup",
    func=lambda city: get_forecast(city),
    description="Useful to get the weather forecast for the next day (Tommorow) for a city.It includes information on temperature, pressure, humidity, wind, clouds and rain."
)

In [7]:

# Load Llama.cpp Model
llm = LlamaCpp(model_path="../models/Phi-3-mini-4k-instruct-gguf/Phi-3-mini-4k-instruct-q4.gguf",n_ctx=4096,n_gpu_layers=-1)  

# Tools (Include both Weather and Forecast Tools)
tools = [weather_tool, forecast_tool]

# Initialize Agent
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True) 


llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from ../models/Phi-3-mini-4k-instruct-gguf/Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.

In [8]:
print(agent.run("What's the forecast for tomorrow in Kalyani?")) 

/home/vatsal-patel/anaconda3/envs/unsloth_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...



llama_print_timings:        load time =     153.49 ms
llama_print_timings:      sample time =       4.99 ms /    35 runs   (    0.14 ms per token,  7012.62 tokens per second)
llama_print_timings: prompt eval time =    5489.27 ms /   252 tokens (   21.78 ms per token,    45.91 tokens per second)
llama_print_timings:        eval time =    2048.11 ms /    34 runs   (   60.24 ms per token,    16.60 tokens per second)
llama_print_timings:       total time =    7563.22 ms /   286 tokens


 I need to get the weather forecast for tomorrow in Kalyani.
Action: ForecastLookup
Action Input: Kalyani
Observation: {'2024-07-05': {'00:00:00': {'temperature': 27.19, 'feels_like': 30.71, 'temp_min': 27.19, 'temp_max': 27.19, 'pressure': 1001, 'humidity': 85, 'weather': 'overcast clouds', 'icon': '04d', 'wind_speed': 3.1, 'wind_deg': 152, 'visibility': 10000, 'pop': 0.14, 'rain': 0, 'clouds': 100}, '03:00:00': {'temperature': 31.54, 'feels_like': 36.59, 'temp_min': 31.54, 'temp_max': 31.54, 'pressure': 1002, 'humidity': 62, 'weather': 'overcast clouds', 'icon': '04d', 'wind_speed': 3.69, 'wind_deg': 203, 'visibility': 10000, 'pop': 0.5, 'rain': 0, 'clouds': 100}, '06:00:00': {'temperature': 34.87, 'feels_like': 40.37, 'temp_min': 34.87, 'temp_max': 34.87, 'pressure': 1000, 'humidity': 50, 'weather': 'overcast clouds', 'icon': '04d', 'wind_speed': 3.59, 'wind_deg': 217, 'visibility': 10000, 'pop': 0.5, 'rain': 0, 'clouds': 97}, '09:00:00': {'temperature': 33.83, 'feels_like': 39.77, 

Llama.generate: prefix-match hit

llama_print_timings:        load time =     153.49 ms
llama_print_timings:      sample time =      23.80 ms /   149 runs   (    0.16 ms per token,  6260.50 tokens per second)
llama_print_timings: prompt eval time =   30400.06 ms /  1208 tokens (   25.17 ms per token,    39.74 tokens per second)
llama_print_timings:        eval time =   11294.11 ms /   148 runs   (   76.31 ms per token,    13.10 tokens per second)
llama_print_timings:       total time =   41830.43 ms /  1356 tokens


 I now know the final answer
Final Answer: The forecast for tomorrow in Kalyani is overcast clouds with a possibility of light rain during the daytime. Temperatures will range from lows in the mid-20s to highs around 35 degrees Celsius. Wind speeds are moderate, and visibility remains excellent throughout the day.

Answer: The forecast for tomorrow in Kalyani is overcast clouds with a possibility of light rain during the daytime. Temperatures will range from lows in the mid-20s to highs around 35 degrees Celsius. Wind speeds are moderate, and visibility remains excellent throughout the day.

> Finished chain.
The forecast for tomorrow in Kalyani is overcast clouds with a possibility of light rain during the daytime. Temperatures will range from lows in the mid-20s to highs around 35 degrees Celsius. Wind speeds are moderate, and visibility remains excellent throughout the day.

Answer: The forecast for tomorrow in Kalyani is overcast clouds with a possibility of light rain during the d

In [9]:
# Example Usage
print(agent.run("Can you tell me current weather in Kalyani?")) 

Llama.generate: prefix-match hit




> Entering new AgentExecutor chain...



llama_print_timings:        load time =     153.49 ms
llama_print_timings:      sample time =       5.25 ms /    33 runs   (    0.16 ms per token,  6285.71 tokens per second)
llama_print_timings: prompt eval time =     410.24 ms /    15 tokens (   27.35 ms per token,    36.56 tokens per second)
llama_print_timings:        eval time =    2053.39 ms /    32 runs   (   64.17 ms per token,    15.58 tokens per second)
llama_print_timings:       total time =    2486.40 ms /    47 tokens


 I need to find out today's weather for Kalyani.
Action: WeatherLookup
Action Input: Kalyani
Observation: {
  "coordinates": {
    "lon": 88.4833,
    "lat": 22.9833
  },
  "weather": {
    "id": 804,
    "main": "Clouds",
    "description": "overcast clouds",
    "icon": "04n"
  },
  "temperature": {
    "current": 26.95,
    "feels_like": 30.39,
    "min": 26.95,
    "max": 26.95
  },
  "pressure": {
    "sea_level": 1001,
    "ground_level": 1000
  },
  "humidity": 88,
  "visibility": 10000,
  "wind": {
    "speed": 3.42,
    "deg": 204,
    "gust": 6.67
  },
  "clouds": {
    "all": 100
  },
  "rain": {},
  "dt": 1720031923,
  "sys": {
    "type": 1,
    "id": 9114,
    "country": "IN",
    "sunrise": 1720049108,
    "sunset": 1720097730
  },
  "timezone": 19800,
  "id": 1268293,
  "name": "Kalyani",
  "cod": 200
}
Thought:

Llama.generate: prefix-match hit

llama_print_timings:        load time =     153.49 ms
llama_print_timings:      sample time =      23.48 ms /   144 runs   (    0.16 ms per token,  6131.83 tokens per second)
llama_print_timings: prompt eval time =   10474.10 ms /   414 tokens (   25.30 ms per token,    39.53 tokens per second)
llama_print_timings:        eval time =    9721.62 ms /   143 runs   (   67.98 ms per token,    14.71 tokens per second)
llama_print_timings:       total time =   20306.78 ms /   557 tokens


 I now know the current weather in Kalyani. It's overcast with a temperature of approximately 27 degrees Celsius and no rain reported. The pressure is around 1001 hPa, humidity is at 88%, wind speed is about 3.4 km/h from the south-west direction.
Final Answer: The current weather in Kalyani is overcast clouds with a temperature of approximately 27 degrees Celsius and no rain reported. The pressure is around 1001 hPa, humidity is at 88%, wind speed is about 3.4 km/h from the south-west direction.

> Finished chain.
The current weather in Kalyani is overcast clouds with a temperature of approximately 27 degrees Celsius and no rain reported. The pressure is around 1001 hPa, humidity is at 88%, wind speed is about 3.4 km/h from the south-west direction.
